In [1]:
import pandas as pd

df = pd.read_json("../data/totto/linearized/train.jsonl", lines=True)

In [2]:
import json

df["table"] = df["table"].apply(lambda x: pd.DataFrame(json.loads(x)))

In [3]:
df["num_cells"] = df["highlighted_cells"].apply(lambda x: len(x))
df["num_highlighted_rows"] = df["highlighted_cells"].apply(lambda x: len(set([c[0] for c in x])))
df["num_highlighted_cols"] = df["highlighted_cells"].apply(lambda x: len(set([c[1] for c in x])))
df["num_rows"] = df["table"].apply(lambda x: len(x))
df["num_cols"] = df["table"].apply(lambda x: len(x.columns))
df["num_numeric_headers"] = df["table"].apply(lambda x: len([c for c in x.columns if c.isdigit()]))

In [13]:
import numpy as np
import regex as re

def numeric_ratio(inputs):
    table, highlighted_cells = inputs
    numeric = 0
    total = 0
    for i, j in highlighted_cells:
        total += 1
        value = re.search('[A-Za-z]', str(table.iloc[i, j]))
        if not value:
            numeric += 1

    if total == 0:
        return 1.0
    return numeric * 1.0 / total

In [12]:
def empty_ratio(table):
    return table.replace("", np.nan).isna().mean().mean()

In [ ]:
from multiprocessing import Pool
tables = df["table"].tolist()
highlighted_cells = df["highlighted_cells"].tolist()
pool = Pool()
df["empty_rate"] = pool.map(empty_ratio, tables)
pool.close()

pool = Pool()
df["numeric_rate"] = pool.map(numeric_ratio, zip(tables, highlighted_cells))
pool.close()

In [8]:
len(df["num_cells"])

120761

In [19]:
len(df[df["num_cells"] < 1])

12470

In [6]:
len(df[df["num_highlighted_rows"] <= 3])

117364

In [24]:
len(df[df["num_highlighted_cols"] > 10])

21

In [23]:
len(df[df["numeric_rate"]  >= 0.99])

31526

In [22]:
len(df[df["empty_rate"]  >= 0.99])

68

In [ ]:
df["num_numeric_headers"] = df["table"].apply(lambda x: len([c for c in x.columns if c.isdigit()]))

In [25]:
len(df[df["num_numeric_headers"] == df["num_cols"]])

12866

In [28]:
len(df[df["table_page_title"].str.contains("List")])

21729

In [26]:
import spacy
nlp = spacy.load("en_core_web_sm")

def filter_sentence(jobj):
    doc = nlp(jobj["sentence_annotations"][0]["original_sentence"])
    for token in doc:
        if token.pos_ == "VERB":
              return True
    return False

df["sentence"] = df["sentence_annotations"].apply(lambda x: x[0]["original_sentence"])

/opt/conda/envs/read/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
pool = Pool()

jobjs = df.to_dict("records")
has_verb = pool.map(filter_sentence, jobjs)
df["has_verb"] = has_verb
df = df[df["has_verb"]]

In [29]:
len(df)

106957

In [48]:
df.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers
count,1.207610e+05,120761.000000,120761.000000,120761.000000,120761.000000,120761.000000,120761.000000
mean,9.972897e+14,3.041802,1.314406,2.385381,28.180414,6.298929,0.442850
std,5.323709e+18,5.485261,3.798645,1.387965,76.151832,4.297104,1.948415
min,-9.223283e+18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-4.612932e+18,2.000000,1.000000,1.000000,5.000000,4.000000,0.000000
50%,-6.394594e+15,3.000000,1.000000,2.000000,12.000000,5.000000,0.000000
75%,4.604847e+18,4.000000,1.000000,3.000000,27.000000,8.000000,0.000000
max,9.223179e+18,640.000000,430.000000,28.000000,3203.000000,76.000000,39.000000


In [79]:
from multiprocessing import Pool
tables = df["table"].tolist()
highlighted_cells = df["highlighted_cells"].tolist()
pool = Pool()
df["empty_rate"] = pool.map(empty_ratio, tables)
pool.close()

pool = Pool()
df["numeric_rate"] = pool.map(numeric_ratio, zip(tables, highlighted_cells))
pool.close()

In [80]:
df.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,1.207610e+05,120761.000000,120761.000000,120761.000000,120761.000000,120761.000000,120761.000000,114486.000000,120761.000000
mean,9.972897e+14,3.041802,1.314406,2.385381,28.180414,6.298929,0.442850,0.166451,0.431918
std,5.323709e+18,5.485261,3.798645,1.387965,76.151832,4.297104,1.948415,0.175553,0.388382
min,-9.223283e+18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-4.612932e+18,2.000000,1.000000,1.000000,5.000000,4.000000,0.000000,0.000000,0.000000
50%,-6.394594e+15,3.000000,1.000000,2.000000,12.000000,5.000000,0.000000,0.111111,0.333333
75%,4.604847e+18,4.000000,1.000000,3.000000,27.000000,8.000000,0.000000,0.285714,1.000000
max,9.223179e+18,640.000000,430.000000,28.000000,3203.000000,76.000000,39.000000,1.000000,1.000000


In [81]:
df1 = df[df["num_cells"] >= 1]
df1.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,1.082910e+05,108291.000000,108291.000000,108291.000000,108291.000000,108291.000000,108291.000000,108291.000000,108291.000000
mean,-3.612531e+14,3.392073,1.465764,2.660064,30.549695,6.574600,0.462328,0.163446,0.366502
std,5.325420e+18,5.688997,3.983651,1.190632,79.564683,3.668501,1.944386,0.169552,0.356046
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.617195e+18,2.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-1.348829e+16,3.000000,1.000000,3.000000,13.000000,5.000000,0.000000,0.111111,0.333333
75%,4.610805e+18,4.000000,1.000000,3.000000,29.000000,8.000000,0.000000,0.275312,0.500000
max,9.223179e+18,640.000000,430.000000,28.000000,3203.000000,65.000000,30.000000,0.961427,1.000000


In [82]:
df2 = df1[df1["num_highlighted_cols"] <= 10]
df2.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,1.082700e+05,108270.000000,108270.000000,108270.000000,108270.000000,108270.000000,108270.000000,108270.000000,108270.000000
mean,-2.050275e+14,3.389212,1.465660,2.657874,30.553671,6.571516,0.461097,0.163439,0.366493
std,5.325341e+18,5.682930,3.983996,1.178626,79.571648,3.657303,1.938958,0.169542,0.356019
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.617134e+18,2.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-1.293415e+16,3.000000,1.000000,3.000000,14.000000,5.000000,0.000000,0.111111,0.333333
75%,4.610780e+18,4.000000,1.000000,3.000000,29.000000,8.000000,0.000000,0.275099,0.500000
max,9.223179e+18,640.000000,430.000000,10.000000,3203.000000,64.000000,30.000000,0.961427,1.000000


In [83]:
df3 = df2[df2["num_highlighted_rows"] <= 3]
df3.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,1.048770e+05,104877.000000,104877.000000,104877.000000,104877.000000,104877.000000,104877.000000,104877.000000,104877.000000
mean,4.944286e+14,3.016581,1.218561,2.660364,30.307389,6.569725,0.466728,0.162686,0.368338
std,5.324233e+18,1.605110,0.491840,1.177731,79.831930,3.657501,1.953127,0.169380,0.356937
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.613079e+18,2.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-6.871971e+15,3.000000,1.000000,3.000000,13.000000,5.000000,0.000000,0.111111,0.333333
75%,4.606838e+18,4.000000,1.000000,3.000000,28.000000,8.000000,0.000000,0.274359,0.500000
max,9.223179e+18,21.000000,3.000000,10.000000,3203.000000,64.000000,30.000000,0.961427,1.000000


In [84]:
df4 = df3[df3["num_numeric_headers"] != df3["num_cols"]]
df4.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,9.849600e+04,98496.000000,98496.000000,98496.000000,98496.000000,98496.000000,98496.000000,98496.000000,98496.000000
mean,3.822672e+15,3.033524,1.216831,2.670687,31.541362,6.660829,0.162453,0.165838,0.376368
std,5.322904e+18,1.607527,0.492722,1.165246,81.942953,3.667843,1.373199,0.170512,0.358327
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.598772e+18,2.000000,1.000000,2.000000,7.000000,4.000000,0.000000,0.000000,0.000000
50%,2.756485e+15,3.000000,1.000000,3.000000,14.000000,6.000000,0.000000,0.114286,0.333333
75%,4.606240e+18,4.000000,1.000000,3.000000,30.000000,8.000000,0.000000,0.281250,0.500000
max,9.223179e+18,21.000000,3.000000,10.000000,3203.000000,64.000000,30.000000,0.961427,1.000000


In [85]:
df4 = df4[~df4["table_page_title"].str.contains("List")]

In [86]:
df4.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,7.422900e+04,74229.000000,74229.000000,74229.000000,74229.000000,74229.000000,74229.000000,74229.000000,74229.000000
mean,4.663737e+15,3.243005,1.253216,2.815571,17.445055,6.747834,0.200016,0.171237,0.440350
std,5.319111e+18,1.641507,0.524634,1.162042,24.695068,3.888854,1.542090,0.172651,0.360936
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.589108e+18,2.000000,1.000000,2.000000,5.000000,4.000000,0.000000,0.000000,0.200000
50%,1.498249e+15,3.000000,1.000000,3.000000,11.000000,5.000000,0.000000,0.116223,0.333333
75%,4.600455e+18,4.000000,1.000000,3.000000,21.000000,8.000000,0.000000,0.300000,0.666667
max,9.223179e+18,21.000000,3.000000,10.000000,1523.000000,64.000000,30.000000,0.948718,1.000000


In [89]:
df5 = df4[df4["numeric_rate"] < 0.99]    
df5.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,5.690400e+04,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000
mean,-5.394198e+15,3.562562,1.285569,3.070347,19.259103,6.408846,0.144665,0.158553,0.269959
std,5.317162e+18,1.583559,0.548406,1.054391,26.938580,3.414428,1.156210,0.167436,0.213411
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.604776e+18,3.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-1.713529e+16,3.000000,1.000000,3.000000,12.000000,5.000000,0.000000,0.111111,0.333333
75%,4.588275e+18,4.000000,1.000000,4.000000,24.000000,8.000000,0.000000,0.250000,0.400000
max,9.223179e+18,21.000000,3.000000,10.000000,1523.000000,64.000000,29.000000,0.948718,0.900000


In [95]:
df6 = df5[df5["empty_rate"] < 0.99]    

In [96]:
df6.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,5.690400e+04,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000,56904.000000
mean,-5.394198e+15,3.562562,1.285569,3.070347,19.259103,6.408846,0.144665,0.158553,0.269959
std,5.317162e+18,1.583559,0.548406,1.054391,26.938580,3.414428,1.156210,0.167436,0.213411
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.604776e+18,3.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-1.713529e+16,3.000000,1.000000,3.000000,12.000000,5.000000,0.000000,0.111111,0.333333
75%,4.588275e+18,4.000000,1.000000,4.000000,24.000000,8.000000,0.000000,0.250000,0.400000
max,9.223179e+18,21.000000,3.000000,10.000000,1523.000000,64.000000,29.000000,0.948718,0.900000


In [97]:
df6["table"]

1           Engine (code)     Displacement, Type Valvetr...
2                                                       ...
3                 State (class)                     Vaca...
4                         Chart (2013) Peak Position
0  ...
7               Party                 Candidate   Votes ...
                                ...                        
120727      Division of Club Apps of Season Goals of Lea...
120729                   Film                         Ro...
120731      Rank Lane             Name    Nationality   ...
120732      Call sign Frequency (MHz)          City of l...
120738           #              Date            Athlete ...
Name: table, Length: 56904, dtype: object

In [105]:
import spacy
nlp = spacy.load("en_core_web_sm")

def filter_sentence(jobj):
    doc = nlp(jobj["sentence_annotations"][0]["original_sentence"])
    for token in doc:
        if token.pos_ == "VERB":
              return True
    return False

df["sentence"] = df["sentence_annotations"].apply(lambda x: x[0]["original_sentence"])

In [ ]:
jobjs = df6.to_dict("records")

In [106]:
pool = Pool()

has_verb = pool.map(filter_sentence, jobjs)
df6["has_verb"] = has_verb
df7 = df6[df6["has_verb"]]

In [108]:
df7.describe()

,example_id,num_cells,num_highlighted_rows,num_highlighted_cols,num_rows,num_cols,num_numeric_headers,empty_rate,numeric_rate
count,5.398100e+04,53981.000000,53981.000000,53981.000000,53981.000000,53981.000000,53981.000000,53981.000000,53981.000000
mean,-1.371650e+16,3.578630,1.289843,3.080399,19.359460,6.430244,0.141772,0.159309,0.270034
std,5.321634e+18,1.582693,0.551516,1.053401,26.845556,3.443135,1.150108,0.167168,0.210812
min,-9.223283e+18,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,-4.621000e+18,3.000000,1.000000,2.000000,6.000000,4.000000,0.000000,0.000000,0.000000
50%,-2.679493e+16,3.000000,1.000000,3.000000,12.000000,5.000000,0.000000,0.111111,0.333333
75%,4.582631e+18,4.000000,1.000000,4.000000,24.000000,8.000000,0.000000,0.250000,0.400000
max,9.223084e+18,21.000000,3.000000,10.000000,1523.000000,64.000000,29.000000,0.948718,0.900000
